In [ ]:
# !unzip '/content/drive/MyDrive/SPAM IMAGE dataset2-20240307T055900Z-001.zip'

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
train_dir='/content/SPAM IMAGE dataset2/train'
validation_dir='/content/SPAM IMAGE dataset2/val'

In [ ]:
img_size=(244,244)
batch_size=32

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 1225 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 349 images belonging to 2 classes.


In [ ]:
# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Combine base model with new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
train_generator.samples

1225

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
38/38 [==============================] - 31s 558ms/step - loss: 0.2107 - accuracy: 0.9480 - val_loss: 0.0668 - val_accuracy: 0.9875
Epoch 2/10
38/38 [==============================] - 22s 573ms/step - loss: 0.0811 - accuracy: 0.9816 - val_loss: 0.0525 - val_accuracy: 0.9937
Epoch 3/10
38/38 [==============================] - 22s 589ms/step - loss: 0.0749 - accuracy: 0.9799 - val_loss: 0.0513 - val_accuracy: 0.9875
Epoch 4/10
38/38 [==============================] - 22s 568ms/step - loss: 0.0635 - accuracy: 0.9816 - val_loss: 0.0498 - val_accuracy: 0.9875
Epoch 5/10
38/38 [==============================] - 22s 572ms/step - loss: 0.0594 - accuracy: 0.9858 - val_loss: 0.0477 - val_accuracy: 0.9875
Epoch 6/10
38/38 [==============================] - 21s 541ms/step - loss: 0.0551 - accuracy: 0.9832 - val_loss: 0.0463 - val_accuracy: 0.9906
Epoch 7/10
38/38 [==============================] - 22s 590ms/step - loss: 0.0482 - accuracy: 0.9866 - val_loss: 0.0453 - val_accuracy: 0.9906

In [ ]:
# Save the model
model.save('spam_image_filter_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('spam_image_filter_model.h5')

In [ ]:
# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
# Function to predict whether the image contains spam or not
def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/SPAM IMAGE dataset2/test/SpamImages/50WgVqGONr.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")


1/1 [==============================] - 1s 1s/step
The uploaded image is predicted as SPAM.
